In [11]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import torch
import numpy as np
from torch_geometric.data import Data
import anndata
# import os
import hdf5plugin
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import numpy as np
np.random.seed(42)  # 예시로 42를 시드로 설정


In [129]:
biogrid = pd.read_csv("/home/hb/lof/network/biogrid/integrated_network_dropdp_sorted_gene_set.csv")


In [182]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# 레이블 인코딩 수행
gene_list = np.array(list(set(np.concatenate([biogrid['gene_symbol_A'].values, biogrid['gene_symbol_B'].values]))))
label_encoder = label_encoder.fit(gene_list)
gene_label = label_encoder.transform(gene_list)
biogrid['label_A'] = label_encoder.transform(biogrid['gene_symbol_A'])
biogrid['label_B'] = label_encoder.transform(biogrid['gene_symbol_B'])
biogrid['edge'] = biogrid['label_A'].astype(str) + '-' + biogrid['label_B'].astype(str)
gene_dict = dict(zip(gene_list, gene_label))
edge_index = torch.tensor(np.vstack([biogrid['label_A'].values, biogrid['label_B'].values]))
biogrid[biogrid['gene_symbol_A']=='MYPN'].head(1)

,gene_symbol_A,gene_symbol_B,label_A,label_B,edge
1,MYPN,ACTN2,7811,172,7811-172


In [202]:
DIM = len(gene_list)
ahr_x1 = pd.read_csv("/data_hdd4/hb/lof/data/deg_augmentation/AHR/AHR_334.csv")
_df = ahr_x1
_df['gene_label'] = _df['gene_name'].map(gene_dict)
_df = _df.dropna(subset=['gene_label'], ignore_index=True).sort_values(by='gene_label')
_df['gene_label'] = _df['gene_label'].astype(int)

x_dict = dict(zip(_df['gene_label'].values, _df.iloc[:, -2].values))
not_in_dict = set(range(DIM)) - set(x_dict.keys())
not_in_dict = dict(zip(not_in_dict, [0]*len(not_in_dict)))
x_dict.update(not_in_dict)
len(x_dict.keys())
x = torch.tensor(list(x_dict.values()), dtype=torch.float32).view(-1, 1)
x.shape

torch.Size([14592, 1])

In [186]:
perturb_set = ['AHR']
perturb_gene_label = [gene_dict[x] for x in perturb_set]
_y = np.zeros(DIM)
for _gene in perturb_gene_label:
    _y[_gene] = 1

In [205]:
x, x.shape

(tensor([[-0.4990],
         [-0.1141],
         [-0.1161],
         ...,
         [ 0.0000],
         [ 0.0000],
         [ 0.0000]]),
 torch.Size([14592, 1]))

In [206]:
edge_index

tensor([[ 7026,  7811,   184,  ..., 12987, 11405, 12126],
        [ 4532,   172,  4567,  ..., 13325, 12126, 11405]])

In [207]:
_y.shape

(14592,)

In [203]:
biogrid_data = Data(x=x, edge_index=edge_index, y=_y)
biogrid_data

Data(x=[14592, 1], edge_index=[2, 743637], y=[14592])